# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [72]:
#Load the csv exported from Part 1
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Hilo,19.73,-155.09,84.2,58,1,11.41,US,1607303597
1,Glendive,47.11,-104.71,36.0,66,1,9.17,US,1607304253
2,Puerto Ayora,-0.74,-90.35,72.0,91,72,3.00,EC,1607303963
3,Norman Wells,65.28,-126.83,1.4,84,90,3.36,CA,1607304161
4,Coquimbo,-29.95,-71.34,57.2,82,75,8.05,CL,1607303908


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [73]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)

In [74]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

#Save to folder
plt.savefig("../Images/heat_map")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
#Narrow down data for ideal condition
#Identify rows with needed data using the loc method
weather_new = cities_df.loc[(cities_df["Max Temp(F)"] >=70) &
                           (cities_df["Wind Speed"] < 10) &
                           (cities_df["Cloudiness"] <= 0)]
weather_new.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
8,Formosa do Rio Preto,-11.05,-45.19,73.06,65,0,1.28,BR,1607304260
71,Acajutla,13.59,-89.83,78.80,57,0,2.01,SV,1607304337
74,Santiago del Estero,-27.80,-64.26,71.44,45,0,9.57,AR,1607304078
131,Takoradze,4.88,-1.76,78.89,83,0,7.18,GH,1607304411
184,Kampot,10.62,104.18,82.40,58,0,9.17,KH,1607304471


In [47]:
#The output has 15 rows and we drop the row that does not meet the criteria
drop_value = weather_new.drop(weather_new.index[4])

#We reset the index to later create a dataframe
new_weather_index = drop_value.reset_index(drop=True)
new_weather_index.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Formosa do Rio Preto,-11.05,-45.19,73.06,65,0,1.28,BR,1607304260
1,Acajutla,13.59,-89.83,78.80,57,0,2.01,SV,1607304337
2,Santiago del Estero,-27.80,-64.26,71.44,45,0,9.57,AR,1607304078
3,Takoradze,4.88,-1.76,78.89,83,0,7.18,GH,1607304411
4,Itupiranga,-5.13,-49.33,78.80,100,0,2.19,BR,1607304514


In [50]:
#New dataframe fitting the criteria 
weather_df = pd.DataFrame(new_weather_index)
weather_df.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Formosa do Rio Preto,-11.05,-45.19,73.06,65,0,1.28,BR,1607304260
1,Acajutla,13.59,-89.83,78.80,57,0,2.01,SV,1607304337
2,Santiago del Estero,-27.80,-64.26,71.44,45,0,9.57,AR,1607304078
3,Takoradze,4.88,-1.76,78.89,83,0,7.18,GH,1607304411
4,Itupiranga,-5.13,-49.33,78.80,100,0,2.19,BR,1607304514


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
# geocoordinates
lat = weather_df["Lat"]
lng = weather_df["Lng"]

target_search = "hotel"
target_radius = 5000


# set up a parameters dictionary
params = {
    "location": f"{lat},{lng}",
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
# run a request using our params dictionary
response = requests.get(base_url, params=params)

location


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
